In [108]:
# Carga de librerias
# ------------------
# Librerias de uso general
import holidays
from google.cloud import storage

# Manejo de datos
import numpy as np
import pandas as pd

import kagglehub
import shutil
import os

import optuna
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import f1_score, make_scorer

from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM  # Importación para OC-SVM
from sklearn.neighbors import LocalOutlierFactor # Importación para LOF

from sklearn.feature_extraction import DictVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import pickle
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.metrics import classification_report
from math import sqrt
import pandas as pd
import numpy as np
from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, TimeSeriesSplit
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import chi2

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Estadística y series temporales
from statsmodels.tsa.seasonal import seasonal_decompose

# Se importan las funciones
from sklearn.metrics import  mean_absolute_error

In [109]:
# ---------- 0. configuración ----------
CSV_PATH = "/Users/angeleduardogamarrarios/Repositorio_UDEM/MLops_AMICO/data/costs.csv"       # ajusta si hace falta
TEST_DAYS = 60               # últimos N días para test
RND_ITER = 12                # RandomizedSearchCV iteraciones (ajusta)
TS_SPLITS = 3                # TimeSeriesSplit folds (ajusta)
PERM_REPEATS = 10            # permutación (ajusta)


In [110]:
# ---------- 1. carga y limpieza básica ----------
df = pd.read_csv(CSV_PATH)

# eliminar fila de Service total y columna de total costs
df = df[df['Service'] != 'Service total']
df = df.drop('Total costs($)', axis=1)

# convertir columna de fecha en indice
df['Service'] = pd.to_datetime(df['Service'])
df = df.rename(columns={'Service':'date'}).sort_values('date').set_index('date')

# forzar numérico y revisar columnas
df = df.apply(pd.to_numeric, errors='coerce')
num_cols = df.columns.tolist()

print("Filas:", len(df), "Columnas:", num_cols)

Filas: 351 Columnas: ['Relational Database Service($)', 'EC2-Instances($)', 'FSx($)', 'Elastic File System($)', 'EC2-Other($)', 'CloudWatch($)', 'S3($)', 'Elastic Load Balancing($)', 'Backup($)', 'Key Management Service($)', 'DataSync($)', 'Secrets Manager($)', 'Resilience Hub($)']


In [111]:
# ---------- 2. EDA rápido (resumen + skew + patrón semanal) ----------
print("\nResumen estadístico:")
print(df.describe().T)

skewness = df.skew().sort_values(ascending=False)
print("\nSkewness (top):")
print(skewness.head(10))

weekly_mean = df.groupby(df.index.day_name()).mean()
print("\nMedia por día de la semana (muestra):")
print(weekly_mean.head())


Resumen estadístico:
                                count       mean        std           min  \
Relational Database Service($)  351.0  97.240692  87.975006  1.491792e+01   
EC2-Instances($)                329.0  71.525192  25.109641  5.154986e-01   
FSx($)                          351.0  14.678272   1.855596  1.208052e+01   
Elastic File System($)          351.0   8.064240   6.456387  1.786584e+00   
EC2-Other($)                    351.0   6.076750   2.900253  3.117944e-01   
CloudWatch($)                   351.0   4.398582   7.681060  3.109096e-03   
S3($)                           351.0   2.218248   0.787167  1.023371e+00   
Elastic Load Balancing($)       351.0   2.162289   0.004710  2.160000e+00   
Backup($)                       350.0   1.514773   1.210465  2.810710e-01   
Key Management Service($)       351.0   0.260617   0.032317  2.263914e-01   
DataSync($)                     119.0   0.286251   1.415347  6.440000e-08   
Secrets Manager($)              349.0   0.000513   0.0

In [112]:
# ---------- 3. imputación ----------
# Strategy: cambiar a 0 los valores nulos
df_imputed = df.fillna(0)

In [113]:
# ---------- 4. Box-Cox selectivo para normalizar las varibles ----------
# Aplicar Box-Cox solo a columnas muy sesgadas (skew > 1)
skewed_cols = skewness[skewness > 1].index.tolist()
df_bc = df_imputed.copy()

for c in skewed_cols:
    # Box-Cox exige valores > 0. si hay ceros o negativos shift pequeño
    min_val = df_bc[c].min()
    shift = 0.0 if min_val > 0 else abs(min_val) + 1e-6
    try:
        transformed, lam = boxcox(df_bc[c] + shift)
        df_bc[c] = transformed
        print(f"Box-Cox aplicado a {c}, lambda={lam:.4f}")
    except Exception as e:
        # fallback log1p si falla
        df_bc[c] = np.log1p(df_bc[c] + shift)
        print(f"Box-Cox falló en {c}, aplicado log1p")


Box-Cox aplicado a Secrets Manager($), lambda=0.3130
Box-Cox aplicado a DataSync($), lambda=-0.7391
Box-Cox aplicado a Elastic Load Balancing($), lambda=-693.3143
Box-Cox aplicado a Relational Database Service($), lambda=-0.0614
Box-Cox aplicado a CloudWatch($), lambda=0.0712
Box-Cox aplicado a Key Management Service($), lambda=-4.2383


In [114]:
# ---------- 5. normalización por día de la semana ----------
df_bc['day_of_week'] = df_bc.index.day_name()
scaled = df_bc.copy()
features = [c for c in num_cols]  # lista de features reales

for day in scaled['day_of_week'].unique():
    mask = scaled['day_of_week'] == day
    if mask.sum() < 2:
        # si no hay suficientes ejemplos para el día, omitir
        continue
    scaler = StandardScaler()
    scaled.loc[mask, features] = scaler.fit_transform(scaled.loc[mask, features])


In [115]:
scaled

,Relational Database Service($),EC2-Instances($),FSx($),Elastic File System($),EC2-Other($),CloudWatch($),S3($),Elastic Load Balancing($),Backup($),Key Management Service($),DataSync($),Secrets Manager($),Resilience Hub($),day_of_week
date,,,,,,,,,,,,,,
2024-06-01,-0.456626,-0.999276,-1.060436,-0.976153,-0.904894,-0.962807,-1.318657,0.0,-0.984327,-0.972412,-0.648163,-0.224642,0.0,Saturday
2024-06-02,0.220261,-0.997331,-1.056020,-0.962626,-0.910742,-0.623151,-1.343367,0.0,-1.018993,-0.928814,-0.603618,-0.109480,0.0,Sunday
2024-06-03,-0.085621,-0.707250,-1.141054,-0.964218,-0.641159,-0.827714,-1.399405,0.0,-1.017974,-0.664846,-0.610562,-0.778482,0.0,Monday
2024-06-04,-0.522248,-0.594763,-1.144325,-0.978410,-0.754581,-0.802999,-1.403380,0.0,-1.013475,-0.768366,-0.608584,-0.812283,0.0,Tuesday
2024-06-05,-0.557973,-0.516806,-1.178708,-0.970482,-0.709455,-0.875272,-1.423768,0.0,-1.022353,-0.598085,-0.603590,0.323701,0.0,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-13,2.480296,0.424861,0.717311,1.877659,0.043480,1.769287,1.398555,0.0,2.168519,2.003304,-0.608584,0.011502,0.0,Tuesday
2025-05-14,2.456583,0.471894,0.738882,1.847187,0.123543,1.572706,1.398457,0.0,2.109517,1.989241,-0.603590,-0.170495,0.0,Wednesday
2025-05-15,2.490797,0.483870,0.723159,1.870618,0.634860,1.689286,1.386483,0.0,2.156304,1.962093,-0.596131,4.454251,0.0,Thursday


## Etiquetar el dataframe Scaler si es no Outliers

In [116]:
# 1. Identificar columnas numéricas, excluyendo las no numéricas
numerical_cols = scaled.select_dtypes(include=[np.number]).columns

# 2. Extraer los datos numéricos
X_scaled = scaled[numerical_cols].values

# 3. Calcular la matriz de covarianza y su pseudo-inversa (más estable)
covariance_matrix = np.cov(X_scaled.T)
inv_covariance_matrix = np.linalg.pinv(covariance_matrix)  # evita error de matriz singular

# 4. Calcular el vector media
mean_vector = np.mean(X_scaled, axis=0)

# 5. Grados de libertad = número de columnas numéricas
p = len(numerical_cols)

# 6. Umbral Chi-cuadrado (70% de confianza)
threshold = chi2.ppf(0.70, p)
print(f"Umbral Chi-cuadrado (70% confianza): {threshold:.4f}")

# 7. Función para calcular la distancia de Mahalanobis
def mahalanobis_distance(x, mean, inv_cov):
    diff = x - mean
    return np.sqrt(diff.dot(inv_cov).dot(diff.T))

# 8. Calcular las distancias
mahal_distances = [mahalanobis_distance(row, mean_vector, inv_covariance_matrix) for row in X_scaled]

# 9. Agregar las columnas al DataFrame original
scaled['mahalanobis_distance'] = mahal_distances
scaled['is_outlier_mahalanobis'] = np.where(
    scaled['mahalanobis_distance'] >= threshold,
    'Si',
    'No'
)

# 10. Mostrar una vista previa
scaled.head()


Umbral Chi-cuadrado (70% confianza): 15.1187


,Relational Database Service($),EC2-Instances($),FSx($),Elastic File System($),EC2-Other($),CloudWatch($),S3($),Elastic Load Balancing($),Backup($),Key Management Service($),DataSync($),Secrets Manager($),Resilience Hub($),day_of_week,mahalanobis_distance,is_outlier_mahalanobis
date,,,,,,,,,,,,,,,,
2024-06-01,-0.456626,-0.999276,-1.060436,-0.976153,-0.904894,-0.962807,-1.318657,0.0,-0.984327,-0.972412,-0.648163,-0.224642,0.0,Saturday,1.990844,No
2024-06-02,0.220261,-0.997331,-1.056020,-0.962626,-0.910742,-0.623151,-1.343367,0.0,-1.018993,-0.928814,-0.603618,-0.109480,0.0,Sunday,3.358251,No
2024-06-03,-0.085621,-0.707250,-1.141054,-0.964218,-0.641159,-0.827714,-1.399405,0.0,-1.017974,-0.664846,-0.610562,-0.778482,0.0,Monday,2.729085,No
2024-06-04,-0.522248,-0.594763,-1.144325,-0.978410,-0.754581,-0.802999,-1.403380,0.0,-1.013475,-0.768366,-0.608584,-0.812283,0.0,Tuesday,2.305157,No
2024-06-05,-0.557973,-0.516806,-1.178708,-0.970482,-0.709455,-0.875272,-1.423768,0.0,-1.022353,-0.598085,-0.603590,0.323701,0.0,Wednesday,2.996318,No


In [117]:
# Define el número de días para la prueba y los días que deseas para el entrenamiento
TEST_DAYS = 30
TRAIN_DAYS_WINDOW = 137  # Días del entrenamiento desde el 1 de diciembre del 2024

# ---------- 6. División train/test temporal y ventana de entrenamiento ----------

# 6.1. Calcular la fecha de división (inicio del conjunto de prueba)
split_date = scaled.index.max() - pd.Timedelta(days=TEST_DAYS)

# 6.2. Calcular la fecha de inicio del entrenamiento
train_start_date = split_date - pd.Timedelta(days=TRAIN_DAYS_WINDOW)

# 6.3. Filtrar el conjunto de entrenamiento:
# Solo los días dentro de la ventana y sin outliers
train_df = scaled[
    (scaled.index >= train_start_date) &
    (scaled.index < split_date) &
    (scaled['is_outlier_mahalanobis'] == 'No')
].drop(columns=['day_of_week','mahalanobis_distance','is_outlier_mahalanobis'])

# 6.4. Filtrar el conjunto de prueba (sin cambios)
test_df = scaled[
    scaled.index >= split_date
].drop(columns=['day_of_week','mahalanobis_distance','is_outlier_mahalanobis'])

# 6.5. Convertir a arrays de numpy
X_train = train_df.values
X_test = test_df.values
cols = train_df.columns.tolist()

# 6.6. Mostrar información
print(f"Fecha de Inicio de Entrenamiento: {train_start_date.date()}")
print(f"Fecha de División (Inicio Test): {split_date.date()}")
print(f"\nTrain shape: {X_train.shape}, Test shape: {X_test.shape}")

Fecha de Inicio de Entrenamiento: 2024-12-01
Fecha de División (Inicio Test): 2025-04-17

Train shape: (137, 13), Test shape: (31, 13)


In [118]:
# ---------- 7. búsqueda de hiperparámetros ----------
# scoring personalizado: media de decision_function (cuanto mayor, mejor)
def scoring_fn(estimator, X, y=None):
    return float(np.mean(estimator.decision_function(X)))

iso = IsolationForest(random_state=42)

param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'max_samples': [0.5, 0.7, 0.9, 'auto'],
    'contamination': [0.001, 0.005, 0.01, 0.02, 0.05],
    'max_features': [0.5, 0.7, 1.0]
}

tscv = TimeSeriesSplit(n_splits=TS_SPLITS)
rnd = RandomizedSearchCV(iso, param_distributions=param_dist, n_iter=RND_ITER,
                         cv=tscv, random_state=42, n_jobs=-1, scoring=scoring_fn, verbose=1)
rnd.fit(X_train)

print("\nMejores parámetros (RandomizedSearch):")
print(rnd.best_params_)

# ajustar pequeño grid alrededor del mejor para refinar (GridSearch)
best = rnd.best_params_
grid = {
    'n_estimators': sorted(list({max(10, best['n_estimators']-50), best['n_estimators'], best['n_estimators']+50})),
    'max_samples': sorted(list(set([best['max_samples'] if best['max_samples']=='auto' else max(0.1, best['max_samples']-0.1), best['max_samples'], min(1.0, best['max_samples']+0.1)]))),
    'contamination': sorted(list({max(0.0005, best['contamination']/2), best['contamination'], min(0.1, best['contamination']*2)})),
    'max_features': sorted(list({max(0.1, best['max_features']-0.2), best['max_features'], min(1.0, best['max_features']+0.2)}))
}
# limpiar valores inválidos
grid['max_samples'] = [v for v in grid['max_samples'] if (isinstance(v, str) or (isinstance(v, float) and 0 < v <= 1))]

gsearch = GridSearchCV(IsolationForest(random_state=42), param_grid=grid, cv=tscv, n_jobs=-1, scoring=scoring_fn, verbose=1)
gsearch.fit(X_train)

print("\nMejores parámetros (GridSearch):")
print(gsearch.best_params_)

best_model = gsearch.best_estimator_

Fitting 3 folds for each of 12 candidates, totalling 36 fits

Mejores parámetros (RandomizedSearch):
{'n_estimators': 50, 'max_samples': 0.9, 'max_features': 0.7, 'contamination': 0.001}
Fitting 3 folds for each of 81 candidates, totalling 243 fits

Mejores parámetros (GridSearch):
{'contamination': 0.0005, 'max_features': 0.8999999999999999, 'max_samples': 0.9, 'n_estimators': 10}


In [119]:
# ---------- 8. predecir y marcar anomalías ----------
test_scores = best_model.decision_function(X_test)   # mayor => más normal
test_pred = best_model.predict(X_test)                # 1 normal, -1 anomalía
test_anomaly = np.where(test_pred == 1, 0, 1)         # 1 = anomalía (más intuitivo)

test_out = test_df.copy()
test_out['anomaly_score'] = test_scores
test_out['anomaly'] = test_anomaly

print("\nAnomalías en test (conteo):", int(test_out['anomaly'].sum()))
print(test_out[['anomaly_score','anomaly']].head(10))

# ---------- 9. importancia de features (permutación) ----------
# Usamos como 'y' las scores del modelo en train y medimos R2 del estimator.decision_function
y_train_scores = best_model.decision_function(X_train)

def scoring_fn_r2(estimator, X, y):
    return r2_score(y, estimator.decision_function(X))

perm = permutation_importance(best_model, X_train, y_train_scores, scoring=scoring_fn_r2,
                              n_repeats=PERM_REPEATS, random_state=42, n_jobs=-1)
perm_importances = pd.Series(perm.importances_mean, index=cols).sort_values(ascending=False)

print("\nTop features por importancia (permutación):")
print(perm_importances.head(10))





Anomalías en test (conteo): 13
            anomaly_score  anomaly
date                              
2025-04-17       0.058717        0
2025-04-18       0.057219        0
2025-04-19       0.071162        0
2025-04-20       0.075622        0
2025-04-21       0.027817        0
2025-04-22       0.048323        0
2025-04-23       0.035377        0
2025-04-24       0.029027        0
2025-04-25       0.029027        0
2025-04-26       0.069269        0

Top features por importancia (permutación):
EC2-Other($)                      1.084951
EC2-Instances($)                  0.482166
S3($)                             0.462942
CloudWatch($)                     0.334881
DataSync($)                       0.310419
FSx($)                            0.285970
Key Management Service($)         0.255926
Relational Database Service($)    0.227309
Backup($)                         0.176092
Secrets Manager($)                0.135938
dtype: float64


In [120]:
# ---------- 10. exportar resultados ----------
test_out.to_csv("test_anomaly_results.csv")
perm_importances.to_csv("feature_importances_permutation.csv")
print("\nResultados guardados: test_anomaly_results.csv, feature_importances_permutation.csv")


Resultados guardados: test_anomaly_results.csv, feature_importances_permutation.csv


In [121]:
import joblib
# Guardar el modelo entrenado
joblib.dump(best_model, 'modelo_entrenado_amico_1.pkl')
modelo_cargado = joblib.load('modelo_entrenado_amico_1.pkl')
pred = modelo_cargado.predict(X_test)
print(pred)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1  1 -1 -1 -1  1  1 -1
  1 -1 -1  1 -1 -1 -1]
